In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import requests 
import json
from pandas import json_normalize
from geopy.geocoders import Nominatim
import seaborn as sns

In [2]:
def tiempazo(ciudad):
    
    #sacamos coordenadas
    geolocator = Nominatim(user_agent="edu")
    location = geolocator.geocode(f'{ciudad}')
    lat = location.latitude
    lon = location.longitude
    
    #usamos api
    api_key = os.getenv("Api_key")
    url_tiempo = f"https://api.tutiempo.net/json/?lan=es&apid={api_key}&ll={lat},{lon}"
    response = requests.get (url =  url_tiempo).json()
    
    dias_lista = ["day1","day2","day3","day4","day5","day6","day7"]
    dias = []
    for k,v in response.items():
        if k in dias_lista:
            dias.append(v)
            
    df =json_normalize(dias)
    df = df.drop(["moonrise","moonset","moon_phases_icon"], axis = 1)
    
    #añadimos emojis
    dicc = {1:"☀️",2:"🌤",4:"⛅️",6:"🌥",7:"☁️",9:"☁️",11:"💨",18:"🌧",19:"🌧",21:"⛈",22:"⛈",24:"🌧",25:"🌪",28:"🌨",29:"🌨",30:"🌨",33:"🌦",51:"🌦",54:"🌫"}

    df["icon"] = df["icon"].astype(int)
    df["icon"] = df["icon"].map(dicc)

    return df

In [3]:
tiempo = tiempazo("barcelona")
tiempo

,date,temperature_max,temperature_min,icon,text,humidity,wind,wind_direction,icon_wind,sunrise,sunset
0,2021-5-18,20,14,☁️,Cubierto,77,31,Suroeste,SO,6:32,21:04
1,2021-5-19,18,15,☁️,Cubierto,74,27,Suroeste,SO,6:31,21:05
2,2021-5-20,20,14,☁️,Cubierto,78,20,Suroeste,SO,6:30,21:06
3,2021-5-21,20,15,☁️,Cubierto,79,23,Suroeste,SO,6:29,21:07
4,2021-5-22,18,16,☁️,Cubierto,78,16,Sureste,SE,6:28,21:08
5,2021-5-23,19,17,☁️,Cubierto,62,23,Sureste,SE,6:27,21:09
6,2021-5-24,18,16,☁️,Cubierto,63,11,Sureste,SE,6:27,21:10


In [7]:
import plotly.express as px 
import pandas as pd 
import numpy as np 

# data
fig = px.line(tiempo, x=tiempo.date, y=tiempo.columns[1:3],template= "ggplot2")
# Show plot 
fig.show()